In [0]:
from tqdm import tqdm
import cv2
import pandas as pd
import os
import random
import time
import imutils
import math
import matplotlib.pyplot as plt

# from tensorflow.keras.applications.resnet50 import ResNet50
# from keras.applications.resnet50 import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint


from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.models import model_from_json
from tensorflow.keras.optimizers import Adam

In [0]:
def get_labels(file):
    with open(file,'r') as f:
        lines = f.readlines()
        return np.array([line.strip().split() for line in lines])
    
def index_process(flo):
    below = math.floor(flo)
    top = math.ceil(flo)
    
    result = np.zeros(5)
    result[below-1] = top-round(flo)
    result[top-1] = round(flo)-below
    
    return result

#loading of images and labels
#Cross validation was made by the dataset provider
TRAIN_TEST_PATH = './train_test_files/'
train_labels = get_labels(TRAIN_TEST_PATH+'train.txt')
test_labels = get_labels(TRAIN_TEST_PATH+'test.txt')

PATH_IMAGES = './Images/'
X_pre_train = [cv2.imread(PATH_IMAGES+file,0) for file in tqdm(train_labels[:,0])]
X_train = np.array([cv2.resize(img,(224,224),interpolation=cv2.INTER_AREA) for img in X_pre_train])

X_pre_test = [cv2.imread(PATH_IMAGES+file,0) for file in tqdm(test_labels[:,0])]
X_test = np.array([cv2.resize(img,(224,224),interpolation=cv2.INTER_AREA) for img in X_pre_test])

y_train = np.array([float(flo) for flo in train_labels[:,1]])
y_test = np.array([float(flo) for flo in test_labels[:,1]])

X_train = X_train.reshape((X_train.shape[0],X_train.shape[1],X_train.shape[2],1))
X_test = X_test.reshape((X_test.shape[0],X_test.shape[1],X_test.shape[2],1))
X_train.shape,y_train.shape

100%|██████████| 2200/2200 [15:33<00:00,  2.36it/s]


((3300, 224, 224, 1), (3300,))

In [0]:
# def euclidean_distance_loss(y_true, y_pred):
#     """
#     Euclidean distance loss
#     https://en.wikipedia.org/wiki/Euclidean_distance
#     :param y_true: TensorFlow/Theano tensor
#     :param y_pred: TensorFlow/Theano tensor of the same shape as y_true
#     :return: float
#     """
#     return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

In [0]:
# resnet = ResNet50(include_top=False, pooling='avg', input_shape=((224,224,3)))

In [0]:
# model = Sequential()
# model.add(resnet)
# model.add(Dropout(0.5))
# model.add(Dense(5, activation='softmax'))
# model.layers[0].trainable = False
# print (model.summary())

In [0]:
class Beau:
    def __init__(self):

        self.model = Sequential()

        #1st 2dConvolutional Layer
        self.model.add(Conv2D(64, (3, 3), padding='same', input_shape=(224, 224, 1)))
        self.model.add(Activation('relu'))

        #1st 2dMaxPool Layer
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.5))

        #2nd 2dConvolutional Layer
        self.model.add(Conv2D(64, (3, 3), padding='same'))
        self.model.add(Activation('relu'))

        #2nd 2dMaxPool Layer
        self.model.add(MaxPooling2D(pool_size=(2, 2)))  
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.25))

        #3rd 2dConvolutional Layer
        self.model.add(Conv2D(128, (5, 5), padding='same'))
        self.model.add(Activation('relu'))

        #3rd 2dMaxPool Layer
        self.model.add(MaxPooling2D(pool_size=(2, 2)))  
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.5))
        
        #4th 2dConvolutional Layer
        self.model.add(Conv2D(128, (5, 5), padding='same'))
        self.model.add(Activation('relu'))

        #3rd 2dMaxPool Layer
        self.model.add(MaxPooling2D(pool_size=(2, 2)))  
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.25))
        
        #5th 2dConvolutional Layer
        self.model.add(Conv2D(256, (3, 3), padding='same'))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())

        #3rd 2dMaxPool Layer
        self.model.add(MaxPooling2D(pool_size=(2, 2)))  
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.25))
        
        #6th 2dConvolutional Layer
        self.model.add(Conv2D(512, (5, 5), padding='same'))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())

        self.model.add(MaxPooling2D(pool_size=(1, 1)))  
        self.model.add(BatchNormalization())
        self.model.add(Dropout(0.5))

        self.model.add(Conv2D(1024, (5, 5), padding='same'))
        self.model.add(Activation('relu'))
        self.model.add(BatchNormalization())

        self.model.add(Flatten())

        #1st FC Layer
        self.model.add(Dense(256))
        self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.25))
        
        #2nd FC Layer
        self.model.add(Dense(512))
        self.model.add(BatchNormalization())
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.25))

        # Output layer
        self.model.add(Dense(1))
        self.model.add(Activation('relu'))

        # self.model.summary()

In [7]:
model = Beau().model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      640       
_________________________________________________________________
activation (Activation)      (None, 224, 224, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 64)      256       
_________________________________________________________________
dropout (Dropout)            (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      36928     
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 64)      0

In [10]:
sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='kld', optimizer=sgd, metrics=['accuracy'])
# # model.compile(loss=euclidean_distance_loss, optimizer=sgd, metrics=['accuracy'])
# root_mean_squared_error = root_mean_squared_error
model.compile(optimizer=sgd, loss=root_mean_squared_error, metrics=root_mean_squared_error)
earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0, mode='min')
filepath = "{epoch:02d}-{val_loss:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

history = model.fit(X_train, y_train, callbacks=[earlyStopping, checkpoint],
            batch_size=128,
            epochs=100,
            verbose=1,
            validation_data=(X_test, y_test))

Epoch 1/100
26/26 [==============================] - ETA: 0s - loss: 0.4975 - root_mean_squared_error: 0.4972
Epoch 00001: val_loss improved from inf to 0.57123, saving model to 01-0.57.h5
26/26 [==============================] - 19s 738ms/step - loss: 0.4975 - root_mean_squared_error: 0.4972 - val_loss: 0.5712 - val_root_mean_squared_error: 0.5761
Epoch 2/100
26/26 [==============================] - ETA: 0s - loss: 0.4908 - root_mean_squared_error: 0.4912
Epoch 00002: val_loss improved from 0.57123 to 0.56050, saving model to 02-0.56.h5
26/26 [==============================] - 19s 729ms/step - loss: 0.4908 - root_mean_squared_error: 0.4912 - val_loss: 0.5605 - val_root_mean_squared_error: 0.5651
Epoch 3/100
26/26 [==============================] - ETA: 0s - loss: 0.4737 - root_mean_squared_error: 0.4744
Epoch 00003: val_loss improved from 0.56050 to 0.55987, saving model to 03-0.56.h5
26/26 [==============================] - 19s 732ms/step - loss: 0.4737 - root_mean_squared_error: 0.4

In [1]:
plt.figure(figsize=(10,7))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['validation','training'])
plt.title('Loss')
plt.xlabel('Epoch')

In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json1", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./best_model.h5")
print("Saved model to disk")

Saved model to disk


In [39]:
import imutils
color = [(255,0,0),(0,255,255),(0,255,0),(0,100,255),(0,0,255)][::-1]
imagepath = '/content/sample_data/th.jpg'
face_clf = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')
Model = Beau()
Model.model.load_weights('./best_model_224.h5')
image_name = imagepath.split('/')[-1]

img = cv2.imread(imagepath)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
faces = face_clf.detectMultiScale(gray, 1.3, 5)

for (x, y, w, h) in faces:
    fc = gray[y:y+h, x:x+w]

    roi = cv2.resize(fc, (224, 224))
    roi = roi.reshape((1,224,224,1))
    pred = Model.model.predict(roi)[0] 
    cv2.putText(img, str(pred), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
    cv2.rectangle(img,(x,y),(x+w,y+h),color[math.floor(pred)],2)
    plt.figure(figsize=(15,15))
    plt.imshow(imutils.opencv2matplotlib(img))

print(pred)
# cv2.imwrite('out'+image_name,img)

[3.761266]


In [0]:
img.shape

(2000, 2000, 3)